# Test Frame Sender

This file is used for sending individual frames to the device. It measures the accuracy based on individual frames.

**Note that the software on the MCU must match this, and not classify based on clusters of frames.**

In [1]:
import numpy as np
import pandas as pd
import gc

In [13]:
df = pd.read_csv("dataset.csv", header=None)

In [14]:
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float32)            # All but last column as float32
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string
del df
gc.collect()

791

In [4]:
dataset = dataset.reshape(dataset.shape[0], 40, 16, 1)
input_shape = dataset[0].shape

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {input_shape}")

Dataset shape: (583554, 40, 16, 1)
Labels shape: (583554,)
Input shape: (40, 16, 1)


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.05, random_state=42, stratify=labels_set)
del dataset, labels_set
gc.collect()

0

In [6]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))

2025-03-28 09:45:48.353197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743151548.370835  109953 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743151548.375794  109953 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 09:45:48.394364: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/fridh/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. 

In [7]:
def indexOfMax(arr):
    return np.argmax(arr)

In [8]:
import serial
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM0',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=115200,       # Set baud rate to 115200
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1              # Set timeout for reading
)

Configuring serial port...


In [9]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM0 opened at 115200 baud.


In [10]:
def calc_accuracy(y_true, y_pred):
    success = 0
    failure = 0
    for i in range(len(y_pred)):
        if int(y_pred[i]) == int(indexOfMax(y_true[i])):
            success += 1
        else:
            failure += 1
    res = round(success / (success + failure), 4)
    return res

In [ ]:
import struct

x = x_test
y = y_test[0:len(x)]
print(x.shape)

results = []
total = len(x)

# Send data
for i in range(total):
    data = x[i].flatten()
    for j in range(len(data)):
        data_to_send = struct.pack('f', data[j])
        ser.write(data_to_send)
    # No need for sleep here as the delay is already managed by the serial communication timeout.
    # Read response
    response = ser.readline().decode('utf-8').strip()
    results.append(response)
    print(f"Guess: {response} - Class: {indexOfMax(y[i])} Accuracy: {calc_accuracy(y, results)} ({i}/{total})")


(29178, 40, 16, 1)
Guess: 2 - Class: 1 Accuracy: 0.0 (0/29178)
Guess: 1 - Class: 1 Accuracy: 0.5 (1/29178)
Guess: 2 - Class: 2 Accuracy: 0.6667 (2/29178)
Guess: 2 - Class: 3 Accuracy: 0.5 (3/29178)
Guess: 2 - Class: 2 Accuracy: 0.6 (4/29178)
Guess: 0 - Class: 0 Accuracy: 0.6667 (5/29178)
Guess: 3 - Class: 3 Accuracy: 0.7143 (6/29178)
Guess: 1 - Class: 2 Accuracy: 0.625 (7/29178)
Guess: 3 - Class: 1 Accuracy: 0.5556 (8/29178)
Guess: 1 - Class: 2 Accuracy: 0.5 (9/29178)
Guess: 3 - Class: 3 Accuracy: 0.5455 (10/29178)
Guess: 1 - Class: 2 Accuracy: 0.5 (11/29178)
Guess: 1 - Class: 2 Accuracy: 0.4615 (12/29178)
Guess: 0 - Class: 0 Accuracy: 0.5 (13/29178)
Guess: 2 - Class: 1 Accuracy: 0.4667 (14/29178)
Guess: 0 - Class: 1 Accuracy: 0.4375 (15/29178)
Guess: 2 - Class: 2 Accuracy: 0.4706 (16/29178)
Guess: 1 - Class: 2 Accuracy: 0.4444 (17/29178)
Guess: 1 - Class: 1 Accuracy: 0.4737 (18/29178)
Guess: 0 - Class: 2 Accuracy: 0.45 (19/29178)
Guess: 1 - Class: 1 Accuracy: 0.4762 (20/29178)


Guess: 1 - Class: 2 Accuracy: 0.4545 (21/29178)
Guess: 2 - Class: 2 Accuracy: 0.4783 (22/29178)
Guess: 2 - Class: 2 Accuracy: 0.5 (23/29178)
Guess: 1 - Class: 3 Accuracy: 0.48 (24/29178)
Guess: 3 - Class: 2 Accuracy: 0.4615 (25/29178)
Guess: 3 - Class: 1 Accuracy: 0.4444 (26/29178)
Guess: 3 - Class: 3 Accuracy: 0.4643 (27/29178)
Guess: 1 - Class: 1 Accuracy: 0.4828 (28/29178)
Guess: 0 - Class: 1 Accuracy: 0.4667 (29/29178)
Guess: 3 - Class: 3 Accuracy: 0.4839 (30/29178)
Guess: 3 - Class: 3 Accuracy: 0.5 (31/29178)
Guess: 2 - Class: 1 Accuracy: 0.4848 (32/29178)
Guess: 2 - Class: 2 Accuracy: 0.5 (33/29178)
Guess: 1 - Class: 1 Accuracy: 0.5143 (34/29178)
Guess: 3 - Class: 3 Accuracy: 0.5278 (35/29178)
Guess: 2 - Class: 2 Accuracy: 0.5405 (36/29178)
Guess: 1 - Class: 3 Accuracy: 0.5263 (37/29178)
Guess: 0 - Class: 0 Accuracy: 0.5385 (38/29178)
Guess: 0 - Class: 0 Accuracy: 0.55 (39/29178)
Guess: 2 - Class: 2 Accuracy: 0.561 (40/29178)
Guess: 1 - Class: 1 Accuracy: 0.5714 (41/29178)
Guess:

KeyboardInterrupt: 

In [12]:
# Close the serial port
ser.close()